## Import Packages

In [1]:
import struct, csv, json, base64, zlib, pprint, numpy, ctypes, glob
import matplotlib.pyplot as plt
%matplotlib inline

## Relevant Docs

#### [Relevant Python Documentation for Reading C Structs into Python](https://docs.python.org/3.5/library/struct.html "Python Docs for C Structs")

#### [Excellent code snippets for working with Binary data in Python](https://www.devdungeon.com/content/working-binary-data-python "DevDungeon")

|doc | url |
|-----|-----|
| adiBin docs| http://cdn.adinstruments.com/adi-web/manuals/translatebinary/LabChartBinaryFormat.pdf |
| adiBin header|  http://cdn.adinstruments.com/adi-web/manuals/translatebinary/ADIBinaryFormat.h |
| adiBin example|http://cdn.adinstruments.com/adi-web/manuals/translatebinary/TranslateBinary.c |
|source | https://forum.adinstruments.com/viewtopic.php?t=395 |

# Functions

## Base64 to JSON function

In [2]:
def base64ToJson(zippedString):
    json_str = zlib.decompress(base64.b64decode(zippedString)).decode()
    json_json = json.loads(json_str)
    return json_json

## Parse CSV Function

In [3]:
def parseCsv(csv_row, dbg=False):
    
    ############################################################################
    # Parse CSV Data
    ############################################################################
    
    # Record Identifiers 
    patient_id = csv_row[0]
    time_since_admission = csv_row[1]
    
    # Decompress channel data
    channel_json = base64ToJson(row[2])
    
    
    ############################################################################
    # Parse file header and channel header data
    ############################################################################
    
    # Number of channels
    num_channels = len(channel_json)
    
    # Channel titles
    channel_titles = []
    for i in range(num_channels):
        channel_titles.append(channel_json[i]['Label'].upper())
        
    

    ############################################################################
    # Parse channel data
    ############################################################################
    
    # Convert channel data to list
    channel_data = []
    for i in range(num_channels):
        channel_data.append([int(j) for j in channel_json[i]['Text'].split(',')])
        
    
    # Check to see if all channels have the same length
    # If channels are the same length, samples_per_channel set to len element 0
    # If theres a mismatch, pad other channels with mean and
    #   set samples_per_channel to max length
    
    if all(len(i) == len(channel_data[0]) for i in channel_data):
        samples_per_channel = len(channel_data[0])
        if dbg == True:
            print("all channel_data sublists equal length")
    else:
        max_sublist_len = len(max(channel_data, key=len))
        for i in range (num_channels):
            len_dif_tuple = (0, max_sublist_len - len(channel_data[i]))
            channel_data[i] = numpy.pad(channel_data[i]\
                                        , pad_width = len_dif_tuple\
                                        , mode='mean'
                                       )
        samples_per_channel = max_sublist_len  
        if dbg == True:
            print("all channel_data sublists NOT equal length \
                  - padded w/ trailing means")


    ############################################################################
    # Create dictionary of all parsed data from csv file
    ############################################################################
    
    csv_data = {
          'patient_id' : patient_id
        , 'time_since_admission' : time_since_admission
        , 'num_channels' : num_channels
        , 'channel_titles' : channel_titles
        , 'channel_data' : channel_data
        , 'samples_per_channel' : samples_per_channel
    }
    
    
    ############################################################################
    # Debug: Check values
    ############################################################################
    if dbg == True:
        print("patient_id:", patient_id)
        print("time_since_admission:", time_since_admission)
        print("num_channels:", num_channels)
        print("channel_titles:", channel_titles)
        print("samples_per_channel:", samples_per_channel)
        print("\n")
        
    
    ############################################################################
    # Return
    ############################################################################
    return csv_data

## Create ADIBIN Function

In [12]:
def createAdibin(data_dict, dbg=False):
    
    ############################################################################
    # Define default adibin file header variables
    ############################################################################
    
    # Define default file size variables for writing adibin file
    FILE_HEADER_LENGTH = 68
    CHANNEL_HEADER_LENGTH = 96
    CHANNEL_TITLE_LENGTH = 32
    UNITS_LENGTH = 32
    
    # Define format strings for struct
    ADI_FILE_HEADER_FORMAT_STRING = "<4sldlllllddllll"
    ADI_CHANNEL_HEADER_FORMAT_STRING = "<32s32sdddd"
    ADI_CHANNEL_DATA_FORMAT_STRING = "<h"
    
    # Define default file header variables
    magic = b'CFWB'
    version = 1
    secs_per_tick = 1/240
    year = 1776
    month = 7
    day = 4
    hour = 0
    minute = 0
    second = 0
    trigger = 0
    # num_channels passed in dict
    # samples_per_channel passed in dict
    time_channel = 0
    data_format = 3
    
    # Define default channel header variables
    # channel_title passed in dict
    units = {'I':b'mV' \
           , 'II':b'mV' \
           , 'III':b'mV' \
           , 'V':b'mV' \
           , 'AVR':b'mV' \
           , 'AVL':b'mV' \
           , 'AVF':b'mV'  \
           , 'AR2':b'mmHg' \
           , 'SPO2':b'%' \
           , 'RR':b'Imp' \
           , 'RESP':b'Imp'\
           }
    scale = {'I': 2.44 \
           , 'II': 2.44 \
           , 'III': 2.44 \
           , 'V': 2.44 \
           , 'AVR': 2.44 \
           , 'AVL': 2.44 \
           , 'AVF': 2.44  \
           , 'AR2': 0.2 \
           , 'SPO2': 1.0 \
           , 'RR': 0.1 \
           , 'RESP': 0.1 \
           }
    offset = 0
    range_high = 1
    range_low = 0
    # channel_data passed in dict
    
    
    ############################################################################
    # Create units and scales fields from channel titles  passed in dict
    ############################################################################
    
    # Channel units
    channel_units = []
    for i in range(data_dict['num_channels']):
        try:
            channel_units.append(units[data_dict['channel_titles'][i]])
        except:
            channel_units.append('MysteryChannelUnits')
        
    # Channel scales
    channel_scales = []
    for i in range(data_dict['num_channels']):
        try:
            channel_scales.append(scale[data_dict['channel_titles'][i]])
        except:
            channel_scales.append('1.0')
            
   
    ############################################################################
    # Convert channel titles passed in dict to binary strings
    ############################################################################
    
    # New list for binary channel titles
    bin_channel_titles = []
    for i in range(data_dict['num_channels']):
        bin_channel_titles.append(\
                                data_dict['channel_titles'][i].encode('utf-8'))
    
    ############################################################################
    # Append all adibin fields to list of lists
    ############################################################################
    
    # Append file header    
    adibin_header = []
    
    adibin_header.append(magic)
    adibin_header.append(version)
    adibin_header.append(secs_per_tick)
    adibin_header.append(year)
    adibin_header.append(month)
    adibin_header.append(day)
    adibin_header.append(hour)
    adibin_header.append(minute)
    adibin_header.append(second)
    adibin_header.append(trigger)
    adibin_header.append(data_dict['num_channels'])
    adibin_header.append(data_dict['samples_per_channel'])
    adibin_header.append(time_channel)
    adibin_header.append(data_format)
 
    # Append channel headers
    adibin_channel_headers = []

    for i in range(data_dict['num_channels']):
        adibin_channel_headers.append(bin_channel_titles[i])
        adibin_channel_headers.append(channel_units[i])
        adibin_channel_headers.append(channel_scales[i])
        adibin_channel_headers.append(offset)
        adibin_channel_headers.append(range_high)
        adibin_channel_headers.append(range_low)
        
    # Append interleaved channel data
    adibin_channel_data = []
    
    for j in range(data_dict['samples_per_channel']):
        for i in range(data_dict['num_channels']):
            adibin_channel_data.append(data_dict['channel_data'][i][j])
        
    
    
    ############################################################################
    # Pack adibin data into writable struct buffers
    ############################################################################
    
    # Pack file header
    file_header_buffer = bytearray(FILE_HEADER_LENGTH)
    
    struct.pack_into(
        ADI_FILE_HEADER_FORMAT_STRING
        , file_header_buffer
        , 0
        , *adibin_header
        )

    # Pack channel headers
    channel_headers_format_string = ADI_CHANNEL_HEADER_FORMAT_STRING
    for i in range(data_dict['num_channels'] - 1):
        channel_headers_format_string += ADI_CHANNEL_HEADER_FORMAT_STRING[1:]
    
    channel_headers_buffer = bytearray((data_dict['num_channels']) \
                                       * CHANNEL_HEADER_LENGTH)

    struct.pack_into(
        channel_headers_format_string
        , channel_headers_buffer
        , 0
        , *adibin_channel_headers
        )

    # Pack channel data    
    channel_data_format_string = ADI_CHANNEL_DATA_FORMAT_STRING
    for i in range(\
        ((data_dict['num_channels'])*(data_dict['samples_per_channel']))- 1):
        channel_data_format_string += ADI_CHANNEL_DATA_FORMAT_STRING[1:]
    
    channel_data_buffer = bytearray(struct.calcsize(channel_data_format_string))
    
    struct.pack_into(
        channel_data_format_string
        , channel_data_buffer
        , 0
        , *adibin_channel_data
        )

    
    ############################################################################
    # Debug: Check values
    ############################################################################
    if dbg == True:
        print("channel_units:", channel_units)
        print("channel_scales:", channel_scales)
        print("adibin_data[0]:", adibin_data[0])
        print('\n')
        
        
    ############################################################################
    # Return packed adibin buffers
    ############################################################################
    
    # Create dictionary of packed buffers for return
    adibin_data = {'patient_id': data_dict['patient_id']
                  , 'time_since_admission': data_dict['time_since_admission']
                  , 'file_header': file_header_buffer
                  , 'channel_headers': channel_headers_buffer
                  , 'channel_data': channel_data_buffer
                  }
    
    return adibin_data

## Write ADIBIN Function

In [13]:
def writeAdibin(adibin_data, output_directory, dbg=False):
    
    #Create Filename
    filename = output_directory \
                + adibin_data['patient_id'] \
                + "_" \
                + adibin_data['time_since_admission'] \
                + ".adibin"
    
    # Write the returned content to a file in the output directory
    adibin_file = open(filename, 'wb') #overwrite previous file without warning
    adibin_file.write(adibin_data['file_header'])
    adibin_file.write(adibin_data['channel_headers'])
    adibin_file.write(adibin_data['channel_data'])
    adibin_file.close()

    if dbg == True:
        print("Writing:", filename)

## Nice CSV to ADIBIN Function

In [15]:
def csvToAdibin(csv_filename, adibin_out_directory_path, dbg=False):
    
    # Open CSV File
    csv_file = open(csv_filename)
    csv_file_reader = csv.reader(csv_file)
    
    # Parse and write out ADIBIN for every row in the CSV
    for row in csv_file_reader:
        writeAdibin(createAdibin(parseCsv(row, dbg=False), dbg=False)
                   , adibin_out_directory_path
                   , dbg = dbg
                   )

## Sanity Check

In [16]:
csv_file_name = "./sampleFiles/4b80ff2eb1112815299d7a4e9a4a1957.csv"
adibin_directory_path = "./testOutsAdibin/"


csvToAdibin(csv_file_name, adibin_directory_path, dbg=True)


Writing: ./testOutsAdibin/ea6392983e1786204d56da60f29cb2dc_41842.adibin
Writing: ./testOutsAdibin/501d5ed00964868b6115eb924681851e_46596.adibin
Writing: ./testOutsAdibin/4675480aed8a2b6aa25bc49fd262606e_47092.adibin
Writing: ./testOutsAdibin/1b8d5d2955ec9afd70cbddd74cb1fba7_47102.adibin
Writing: ./testOutsAdibin/9b473ecda5b3aa8253521b71db7db565_47112.adibin
Writing: ./testOutsAdibin/1c971c84e514d1c043556a27fe580cb7_47124.adibin
Writing: ./testOutsAdibin/386a268a90055b01306ffab65eadb70d_47132.adibin
Writing: ./testOutsAdibin/dbecd410a1f9606312591328ad29e894_47134.adibin
Writing: ./testOutsAdibin/248f8fa283c65fd69a951f05ece9d00c_47560.adibin
Writing: ./testOutsAdibin/79db2b5bffad61509bf663b3068768b3_44842.adibin
Writing: ./testOutsAdibin/5862f0e233189b4ed8fad491511c5f0d_44864.adibin
Writing: ./testOutsAdibin/64de22cafb7cbdb4cc413ebd2f0066e9_44020.adibin
Writing: ./testOutsAdibin/7cf134c49c429facf53b1bae9a4eddb0_44034.adibin
Writing: ./testOutsAdibin/c0e48c8d7149757c36e39c1450db531b_44038